In [248]:
# ---------------Load Data-----------------
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random as random
# from os import listdir
%matplotlib inline 

# data = pd.read_csv('data/training.csv')
print('libraries imported!')

for i in range(1,0,-1):
    print(i)

libraries imported!
1


# Def getting stock data

In [282]:
stockFiles = os.listdir('stockData')

def getStockData(fileName, size):
    oldStockData = []
    file = open('stockData/' + fileName)
    fLines = file.readlines()
    end = random.randint(1, (len(fLines) - (size*2)+1))
    start = 1200
    end = start + size*2 -1
    print(end-start)
    high = 0
    low = 1000000
    close = float(fLines[start].split(',')[4])
    
    # get the 'old' stock data
    for i in range(end, start-1, -1):
        line = fLines[i].strip()
        lineArray = line.split(',')
        for k in range(1,7):
            lineArray[k] = float(lineArray[k])
        x = lineArray[6]/lineArray[4]

        if(i>=(start+size)):
            print(line)
            for j in range(1,5):
                oldStockData.append(lineArray[j])
            oldStockData.append(lineArray[5])

    
    # get the high/low of 'new' stock data (we try and predict this)
        if(i<(start + size)):
            print(line)
            if (lineArray[2] > high):
                high = lineArray[2]
            if (lineArray[3] < low):
                low = lineArray[3]
    file.close()
    HL = [high,low,close]
    
    return (oldStockData), HL
    
print((getStockData('SPY.txt', 3)))

    

5
2011-10-19,122.379997,123.080002,120.709999,121.129997,226601300,109.78341
2011-10-20,121.43,122.099998,119.82,121.660004,262075600,110.263769
2011-10-21,123.089996,124.120003,122.720001,123.970001,278999400,112.357383
2011-10-24,124.169998,125.800003,124.059998,125.489998,203215600,113.734997
2011-10-25,124.889999,124.949997,122.779999,123.050003,268596800,111.523563
2011-10-26,124.349998,124.769997,122.209999,124.300003,289053800,112.656472
([122.379997, 123.080002, 120.709999, 121.129997, 226601300.0, 121.43, 122.099998, 119.82, 121.660004, 262075600.0, 123.089996, 124.120003, 122.720001, 123.970001, 278999400.0], [125.800003, 122.209999, 124.300003])
